The work below relates to the effective mass theory sections of the 2017 publication _Slow cooling of hot polarons in halide perovskite solar cells_. The notebook was written by Jarvist M. Frost.

In [1]:
# Physics Constants for electronic structure + stat physics
# defined as const to help with type stability
# 2016-07 - Updated accuracy from https://github.com/DataWookie/PhysicalConstants.jl
const c = 2.99792458e8;                            # m / s  # speed of light in this universe
const h= 6.62606896e-34;                           # kg m2 / s 
const hbar = const ħ = 1.05457162825e-34;          # kg m2 / s 

const eV = const q = const ElectronVolt = 1.602176487e-19;                         # kg m2 / s2 
const MassElectron = const melectron = 9.10938188e-31;                          # kg
const ε0 = const ε_0 = const VacuumPermittivity = 8.854187817e-12;                   # A2 s4 / kg m3 

k_b=GSLCONSTMKSABOLTZMANN = 1.3806504e-23 # kg m2 / K s2

# Units
Å=1E-10

1.0e-10

In [2]:
# Material specifics - for MAPbI3
d=6.2Å # lattice spacing, MAPbI

# Dielectric constants
ε_Inf=4.5
ε_S=24.1

f=2.25E12 # Characteristic phonon mode frequency
ω=2*π*f

# Bare-band effective masses.
# From QSGW calc, MAPI
m_eff_e=0.12
m_eff_h=0.15
m_eff = m_eff_e # default to using electron effective mass. No particular reason why - but you've got to choose one!

0.12

In [3]:
# via Feynman 1955
#   http://dx.doi.org/10.1103/PhysRev.97.660
function feynmanalpha(ε_Inf,ε_S,freq,m_eff)
    ω=freq*2*pi #frequency to angular velocity
    # Note to self - you've introduced the 4*pi factor into the dielectric constant; 
    # This gives numeric agreement with literature values, but I'm uncertain of the justification.
    # Such a factor also seems to be necessary for calculation of Exciton binding energies (see below).
    # Maybe scientists in the 50s assumed that the Epsilon subsumed the 4*pi factor?
    α=0.5* (1/ε_Inf - 1/ε_S)/(4*pi*ε_0) * (q^2/(hbar*ω)) * (2*MassElectron*m_eff*ω/hbar)^0.5
    return (α)
end

print("\n Cross check vs. literature values.")
print("\n NaCl Frohlich paper α=",feynmanalpha(2.3, 5.6, (4.9E13/(2*pi)), 1.0)) # should be ~about 5 (Feynman1955)
print("\n CdTe  α=",feynmanalpha(7.1,   10.4,  5.08E12, 0.095)) #Stone 0.39 / Devreese 0.29
print("\n GaAs  α=",feynmanalpha(10.89, 12.9,  8.46E12, 0.063)) # Devreese 0.068

print("\n")
print("\n MAPI  4.5, 24.1, 2.25THz (~75 cm^-1) ; α=",feynmanalpha(4.5,   24.1,  2.25E12,    0.12))

#This value used for further calculations...
α=feynmanalpha(ε_Inf, ε_S ,  f,    m_eff)


 Cross check vs. literature values.
 NaCl Frohlich paper α=5.262322769751378
 CdTe  α=0.3505417171832051
 GaAs  α=0.07081920905063181

 MAPI  4.5, 24.1, 2.25THz (~75 cm^-1) ; α=2.3939407113094093

2.3939407113094093

In [4]:
#α=1.1932891264417276 

println("Bare mass (no polaronic...) ",m_eff)
# By Feynman approximation for polaron mass. See Feynman 1955 for original literature.
#  Also Eqn (9.6.21); p. 955
#  Jones and March, Theoretical solid state physics, Vol 2
println("Effective mass Large polaron limit, *(1+α/6) , ",m_eff*(1+α/6))

println("Effective mass Small polaron limit, *0.02*α^6 , ",m_eff*0.02*α^6)

Bare mass (no polaronic...) 0.12
Effective mass Large polaron limit, *(1+α/6) , 0.1678788142261882
Effective mass Small polaron limit, *0.02*α^6 , 0.45174315859021225


In [5]:
# Large polaron radius via p.942 in Theoretical Solid State Physics (Jones and March), Vol 2
# ISBN: 0486650162
ε_Red=(1/ε_Inf-1/ε_S)
print("ε_Red: ",ε_Red,"\n")

# Not sure if you use dressed polaron effective mass here; or direct band structure one
println("Bare effective mass:$(m_eff) , ε_Red:$(ε_Red)")
m_eff=m_eff*(1+α/6)
print("Rescaled (+α/6) m_eff: $(m_eff))")

# Eqn (9.3.7); p.942; 
#  Jones and March, Theoretical solid state physics, Vol 2
rp=2*pi^2*hbar^2*ε_0*ε_Red^-1/(q*q*MassElectron*m_eff)

println("Large polaron radius")
println("Jones and March, rp = ",rp," m")
println("\t = ",rp/Å," Å = ",rp/d," Lattice units")

V=(4/3)*π * rp^3
println("Volume polaron: $V m (SI), ",V/Å^3, " Å^3  ",V/d^3," unit cells^3")

# Eqn (9.3.8); p.942; 
#  Jones and March, Theoretical solid state physics, Vol 2
W=(0.25*q^2/rp)*ε_Red/ε_0
println("Large Polaron binding energy: $W J ",W/q," eV")

ε_Red: 0.18072844628861226
Bare effective mass:0.12 , ε_Red:0.18072844628861226
Rescaled (+α/6) m_eff: 0.1678788142261882)Large polaron radius
Jones and March, rp = 2.7396722621139184e-9 m
	 = 27.396722621139183 Å = 4.418826229215997 Lattice units
Volume polaron: 8.613594990536387e-26 m (SI), 86135.94990536386 Å^3  361.4176676905939 unit cells^3
Large Polaron binding energy: 4.781232783166302e-20 J 0.2984211054126088 eV


In [6]:
# This from T.D.Schultz 1959, Physical Review, Vol 116, Number 3, pp526--543
# Schultz (2.5a)
#rfa(α)=(3/(0.44*α))^(1/2)
# After re-derivation from Feynman1955 ( see 8-8-2017; Yellow 2017.B Notebook pp.33-34 ), 
# realise this is actually 4/9=0.4444...
rfa(α)=(3/((4/9)*α))^(1/2)
# Also spot this, in the variational form from Feynman in Schultz, you need a term of 
# meff in the scaling factor, as the polaron state is expressed in terms of the effective mass

# Schultz (2.5b)
rfb(α)=3*(π/2)^(1/2)*α
#rfb(α)=3*(π/(2*α))^(1/2)

rfb (generic function with 1 method)

In [7]:
println("Let's try and reproduce Schultz little table below (2.5b).")
println("  α=3, 1.42   ||  α=5, 1.00  ||  α=7, 0.748  ||  α=9, 0.557  ||  α=11, 0.443  ")
for α=3:2:11
    print("α=$α ")
    @printf("Schultz(2.5a) α→0 rf=%.2f (2mω)^1/2 ",rfa(α))
    @printf("Schultz (2.5b) α→∞ rf=%.2f (2mω)^1/2\n",rfb(α))
end
println("Unsure of origin of disagreement. (small α is by far the closest) Schultz table may be numerical integration (of variational problem) results, rather than using the asymtotic Feynman solutions.")


Let's try and reproduce Schultz little table below (2.5b).
  α=3, 1.42   ||  α=5, 1.00  ||  α=7, 0.748  ||  α=9, 0.557  ||  α=11, 0.443  
α=3 Schultz(2.5a) α→0 rf=1.50 (2mω)^1/2 Schultz (2.5b) α→∞ rf=11.28 (2mω)^1/2
α=5 Schultz(2.5a) α→0 rf=1.16 (2mω)^1/2 Schultz (2.5b) α→∞ rf=18.80 (2mω)^1/2
α=7 Schultz(2.5a) α→0 rf=0.98 (2mω)^1/2 Schultz (2.5b) α→∞ rf=26.32 (2mω)^1/2
α=9 Schultz(2.5a) α→0 rf=0.87 (2mω)^1/2 Schultz (2.5b) α→∞ rf=33.84 (2mω)^1/2
α=11 Schultz(2.5a) α→0 rf=0.78 (2mω)^1/2 Schultz (2.5b) α→∞ rf=41.36 (2mω)^1/2
Unsure of origin of disagreement. (small α is by far the closest) Schultz table may be numerical integration (of variational problem) results, rather than using the asymtotic Feynman solutions.


In [8]:
for meff in [m_eff_e,m_eff_h]
    α=feynmanalpha(ε_Inf, ε_S ,  f,    meff)
    println("\nMAPI(meff=$meff): α=$α")

    scale=sqrt(2*meff*melectron*ω )
    
    w=3 ; v=w+(2/9)*α
    println("v=$v w=$w by the Feynman1955 small-α expansion")
    println("Schultz (2.5a) α→0 rf=",rfa(α)," (internal units) = ",rfa(α)*scale," (SI, m)")
    println("Schultz (2.5b) α→∞ rf=",rfb(α)," (internal units) = ",rfb(α)*scale," (SI, m)")
    println("Scale of internal units: ",scale," (SI,m)")
end


MAPI(meff=0.12): α=2.3939407113094093
v=3.5319868247354242 w=3 by the Feynman1955 small-α expansion
Schultz (2.5a) α→0 rf=1.679172028574159 (internal units) = 2.952068697325166e-9 (SI, m)
Schultz (2.5b) α→∞ rf=9.001079212137622 (internal units) = 1.5824348983980366e-8 (SI, m)
Scale of internal units: 1.7580501860978867e-9 (SI,m)

MAPI(meff=0.15): α=2.6765070822960193
v=3.5947793516213373 w=3 by the Feynman1955 small-α expansion
Schultz (2.5a) α→0 rf=1.588062856096852 (internal units) = 3.1214326082540507e-9 (SI, m)
Schultz (2.5b) α→∞ rf=10.063512494599985 (internal units) = 1.9780436229975453e-8 (SI, m)
Scale of internal units: 1.965559861985515e-9 (SI,m)


In [9]:
# Polaron volume / touching

# Polaron radius
rp=26.8e-10 # SI units, metres
# This is from Finite Temperature solution at 300 K for electrons (26.8)
# Volume of a cube with 2*polaron radius to a side
Vcube=(2*rp)^3
# Double count as each excitation generates a (hole) and (electron) polaron
Vcube=2*Vcube

Vcube^-1 # = density per metre cubed, close packed
Vcube^-1 / (100^3) # = density per centimetre cubed

3.246950256514266e18

In [10]:
# EXCITONS
# Following p.973 Jones & March, Theoretical Solid State Phys Vol 2.
# ISBN: 0486650162
# Hygronic like Wannier exciton; Eqn (9.8.10); p.973

ε=ε_Inf * ε_0 * 4*pi 
# term of 4pi appears because in the derivation of Dresselhaus 1956, 
# the Epsilon is used as a general prefactor in the denominator of a Coulomb term in the Hamiltonian

me=m_eff_e * MassElectron
mh=m_eff_h * MassElectron

println("Excitons with ε_Inf:$(ε_Inf)")

# reduced mass - consider using polaron masses here
mu=1/(1/me + 1/mh)


println("Effective masses: me: $me mh: $mh mu: $mu")

En(k,n) = -mu*q^4/(2*ħ^2*ε^2*n^2) + ħ^2*k^2/(2*(me+mh))
a0 = (ε*ħ^2)/(mu*q^2)
d=6.2Å # lattice spacing
println("Exciton radius a0: ",a0," m ",a0/Å," Å ",a0/d," Lattice Spacings\n")

Excitons with ε_Inf:4.5
Effective masses: me: 1.0931258256e-31 mh: 1.366407282e-31 mu: 6.072921253333333e-32
Exciton radius a0: 3.5719462619957756e-9 m 35.71946261995775 Å 5.761203648380283 Lattice Spacings



In [11]:
# From above, exciton radius (SI) is a0
V=(4/3)*π * a0^3
println("Exciton radius a0: ",a0," m ",a0/Å," Å ",a0/d," Lattice Spacings")
println("Volume exciton: $V m (SI), ",V/Å^3, " Å^3  ",V/d^3," unit cells^3")

Exciton radius a0: 3.5719462619957756e-9 m 35.71946261995775 Å 5.761203648380283 Lattice Spacings
Volume exciton: 1.9089887055693506e-25 m (SI), 190898.87055693503 Å^3  800.9922063581914 unit cells^3


In [12]:
# Cross-checked exciton binding energy (figuring out the old-skool units) against this classic paper,
# From http:/dx.doi.org/10.1103/PhysRev.116.573 - Thomas and Hopfield, PhysRev 1959 CdS
# Eb=28 meV; Dielectric = 9.3; Mu = 0.18
# mu=0.18 * melectron
# ε_Inf=9.3
# ε=ε_Inf * ε_0 * 4*pi 
# ---> n: 1 k: 0 En(k,n): -28.34762730330596 meV
#   == Happy Jarvist 8^)

In [13]:
println(" Hydrogenic exciton states @ the gamma point")
k=0.0

for n in 1:10
   println("n: ",n," k: ",k," En(k,n): ",1000*En(k,n)/eV," meV")
end


 Hydrogenic exciton states @ the gamma point
n: 1 k: 0.0 En(k,n): -44.79240010358902 meV
n: 2 k: 0.0 En(k,n): -11.198100025897254 meV
n: 3 k: 0.0 En(k,n): -4.976933344843224 meV
n: 4 k: 0.0 En(k,n): -2.7995250064743136 meV
n: 5 k: 0.0 En(k,n): -1.7916960041435608 meV
n: 6 k: 0.0 En(k,n): -1.244233336210806 meV
n: 7 k: 0.0 En(k,n): -0.9141306143589597 meV
n: 8 k: 0.0 En(k,n): -0.6998812516185784 meV
n: 9 k: 0.0 En(k,n): -0.5529925938714694 meV
n: 10 k: 0.0 En(k,n): -0.4479240010358902 meV


In [14]:
println(" Hydrogenic exciton states.\n n is excitation level; k is crystal momentum.")

# Very messy + not currently used anywhere
#kmax=π/d
#for k in -kmax:kmax/50:kmax
#    for n in 1
#        println("n: ",n," k: ",k," En(k,n): ",1000*En(k,n)/eV," meV")
#    end
#end

#krange=-kmax:kmax/50:kmax
#n=1
#E=[1000*En(k,n)/eV for k in krange]

#using Plots
#gr()
#plot(krange,E,label="Exciton dispersion")

 Hydrogenic exciton states.
 n is excitation level; k is crystal momentum.


In [15]:
# OK, time to go fetch the phonons - these data are frequencies (not reduced) in THz

In [16]:
# via 
# jarvist@titanium:~/phonopy-work/2017-03-Scott-Distortions/0010-CsPbI3
# > grep frequency modulation.yaml  | uniq | sed s/frequency://
CsPbI3=[      
    #-0.7638364901 # this mode is soft; but including it will mess the summation!
      0.110239
      0.2554009591
      0.7251139061
      2.5411013872]

4-element Array{Float64,1}:
 0.110239
 0.255401
 0.725114
 2.5411  

In [17]:
# OK, real data here, frequencies - For MAPI
MAPI=[ #    -0.0000005420
    #  0.0000001842
    #  0.0000004295 # acoustic modes
      0.5601007762
      0.7999497115
      0.8624353643
      0.9780432985
      0.9833140605
      1.0374436221
      1.7466045486
      1.9719855248
      2.0680970668
      2.2483954169
      2.4285633345
      2.7486672126
      3.5138222502
      3.8354218617
    3.9861276966]

PhononFreqs=MAPI # for work further down below

15-element Array{Float64,1}:
 0.560101
 0.79995 
 0.862435
 0.978043
 0.983314
 1.03744 
 1.7466  
 1.97199 
 2.0681  
 2.2484  
 2.42856 
 2.74867 
 3.51382 
 3.83542 
 3.98613 

In [18]:
PhononEnergy = PhononFreqs.*1E12* 2*π * ħ # simply ħω ; including unit conversions
println("MAPI Phonon energies, in meV: ",1000*PhononEnergy/q)
println("Sum of (cage mode, 0-15) MAPI phonon energies, in meV: ",sum(1000*PhononEnergy/q))

MAPI Phonon energies, in meV: [2.31639, 3.30833, 3.56675, 4.04486, 4.06666, 4.29052, 7.22338, 8.15548, 8.55296, 9.29862, 10.0437, 11.3676, 14.532, 15.862, 16.4853]
Sum of (cage mode, 0-15) MAPI phonon energies, in meV: 123.11456400472724


In [19]:
T=300
Z=sum(exp.(-PhononEnergy/(k_b*T)))
# Now, we've just calculated the partition function (Z) for a single unit cell, at a certain temperature

11.08608935565894

In [20]:
# Boltzmann / Thermodynamic occupation, (assuming fermi statistics?)
ThermalOccupation=exp.(-PhononEnergy/(k_b*T))/Z

15-element Array{Float64,1}:
 0.0824723
 0.0793678
 0.0785784
 0.0771385
 0.0770735
 0.0764089
 0.0682141
 0.0657985
 0.0647945
 0.0629523
 0.0611638
 0.0581106
 0.0514156
 0.0488373
 0.0476739

In [21]:
sum(ThermalOccupation) # Sanity check - partition function should normalise this to 1

1.0

In [22]:
ThermalOccupation.*PhononEnergy
println("(Average Thermal) Energy in phonons (Boltzmann stats) of unitcell at T= $T, ", 1000/q*ThermalOccupation.*PhononEnergy, " meV")
println("Sum: ",sum(1000/q*ThermalOccupation.*PhononEnergy)," meV")

(Average Thermal) Energy in phonons (Boltzmann stats) of unitcell at T= 300, [0.191038, 0.262574, 0.280269, 0.312014, 0.313432, 0.327834, 0.492736, 0.536618, 0.554185, 0.58537, 0.614313, 0.660576, 0.747172, 0.774659, 0.785919] meV
Sum: 7.438708995926876 meV


In [23]:
println("BE thermal occupation of phonon states...")
BE(T,En)=1./(exp.(En/(k_b*T)) - 1.0) # Bose-Einstein Occupation

println("Test for k_B T=25 meV; i.e. thermal energy phonon occupation at STP.")
BE(300,25E-3*q)

BE thermal occupation of phonon states...
Test for k_B T=25 meV; i.e. thermal energy phonon occupation at STP.


0.6134392648438429

In [24]:
T=300

BEThermalOccupation=BE(T,PhononEnergy)

# This is quite interesting to know in general - average phonon occupation statistics at 300 K for MAPI


15-element Array{Float64,1}:
 10.6679 
  7.32489
  6.75957
  5.90436
  5.87017
  5.53921
  3.10219
  2.69614
  2.5501 
  2.31011
  2.10624
  1.81072
  1.32557
  1.18062
  1.12097

In [25]:
println("(Average Thermal) Energy in phonons (BE stats) of unitcell at T= $T, ", 1000/q*BEThermalOccupation.*PhononEnergy, " meV")
println("Sum: ",sum(1000/q*BEThermalOccupation.*PhononEnergy)," meV")

(Average Thermal) Energy in phonons (BE stats) of unitcell at T= 300, [24.7111, 24.2331, 24.1097, 23.8823, 23.872, 23.7661, 22.4083, 21.9883, 21.8109, 21.4808, 21.1545, 20.5834, 19.2632, 18.727, 18.4795] meV
Sum: 330.4704107906226 meV


In [26]:
Ts=[]
SHCs=[]

for T in 0:20:3000
    print("Temperature $T ")
    BEThermalOccupation=BE(T,PhononEnergy)
    SHC=sum(1000/q*BEThermalOccupation.*PhononEnergy)/T
    println(" BE Phonon E: ",sum(1000/q*BEThermalOccupation.*PhononEnergy)," meV SHC: ",sum(1000/q*BEThermalOccupation.*PhononEnergy)/T," meV/K")
    
    append!(Ts,T)
    append!(SHCs,SHC)
end

Temperature 0  BE Phonon E: 0.0 meV SHC: NaN meV/K
Temperature 20  BE Phonon E: 3.4786027554348755 meV SHC: 0.17393013777174376 meV/K
Temperature 40  BE Phonon E: 16.62831330625127 meV SHC: 0.4157078326562817 meV/K
Temperature 60  BE Phonon E: 35.38498771320442 meV SHC: 0.5897497952200736 meV/K
Temperature 80  BE Phonon E: 56.96727972004571 meV SHC: 0.7120909965005714 meV/K
Temperature 100  BE Phonon E: 80.03317450812287 meV SHC: 0.8003317450812287 meV/K
Temperature 120  BE Phonon E: 103.94352106046085 meV SHC: 0.8661960088371737 meV/K
Temperature 140  BE Phonon E: 128.37194699946122 meV SHC: 0.9169424785675802 meV/K
Temperature 160  BE Phonon E: 153.13839696345636 meV SHC: 0.9571149810216022 meV/K
Temperature 180  BE Phonon E: 178.13658203554803 meV SHC: 0.9896476779752669 meV/K
Temperature 200  BE Phonon E: 203.30012127489843 meV SHC: 1.0165006063744921 meV/K
Temperature 220  BE Phonon E: 228.58557888287612 meV SHC: 1.0390253585585278 meV/K
Temperature 240  BE Phonon E: 253.963408041

In [27]:
# OK, that's more or less linear in temperature - as the phonon modes become populated, the specific heat capacity --> a constant
# So it may be just as easy to calculate specific heat capcity in phonopy and use those figures
using Plots
plot(Ts,SHCs,lab="BE Sum")
plot!(ylab="Specific Heat Capacity (meV/K/unitcell)")
plot!(xlab="Temperature (K)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 3000 
 
 
 0.5 
 
 
 1.0 
 
 
 Temperature (K) 
 
 
 Specific Heat Capacity (meV/K/unitcell) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 58.6982,372.441 62.2782,292.185 65.8583,234.414 69.4383,193.804 73.0184,164.514 76.5984,142.651 80.1785,125.806 83.7585,112.471 87.3386,101.672 90.9186,92.7588 
 94.4987,85.2819 98.0787,78.9235 101.659,73.4518 105.239,68.6947 108.819,64.5216 112.399,60.8316 115.979,57.5458 119.559,54.6014 123.139,51.9481 126.719,49.5449 
 130.299,47.358 133.879,45.3597 137.459,43.5265 141.039,41.8389 144.619,40.2802 148.199,38.8362 151.78,37.4948 155.36,36.2453 158.94,35.0787 162.52,33.9869 
 166.1,32.9631 169.68,32.001 173.26,31.0953 176.84,30.2412 180.42,29.4343 184,28.6709 187.58,27.9475 191.16,27.2611 194.74,26.6089 198.32,25.9884 
 201.9,25.3974 205.48,24.8338 209.06,24.2957 212.64,23.7815 216.22,23.2896 219.801,22.8186 223.381,22.3671 226.961,21.9341 230.541,21.5183 234.121,21.1188 
 237.701,20.7346 241.281,20.3649 244.861,20.0089 248.441,19.6658 252.021,19.3349 255.601,19.0156 259.181,18.7073 262.761,18.4094 266.341,18.1215 269.921,17.8429 
 273.501,17.5734 277.081,17.3124 280.661,17.0595 284.241,16.8144 287.822,16.5767 291.402,16.3461 294.982,16.1223 298.562,15.905 302.142,15.6938 305.722,15.4886 
 309.302,15.2891 312.882,15.0951 316.462,14.9063 320.042,14.7225 323.622,14.5435 327.202,14.3692 330.782,14.1994 334.362,14.0339 337.942,13.8724 341.522,13.715 
 345.102,13.5614 348.682,13.4115 352.262,13.2652 355.843,13.1223 359.423,12.9827 363.003,12.8464 366.583,12.7131 370.163,12.5828 373.743,12.4554 377.323,12.3309 
 380.903,12.209 384.483,12.0897 388.063,11.973 391.643,11.8587 395.223,11.7468 398.803,11.6372 402.383,11.5299 405.963,11.4247 409.543,11.3216 413.123,11.2206 
 416.703,11.1215 420.283,11.0244 423.864,10.9291 427.444,10.8356 431.024,10.7439 434.604,10.654 438.184,10.5656 441.764,10.4789 445.344,10.3938 448.924,10.3102 
 452.504,10.2281 456.084,10.1475 459.664,10.0682 463.244,9.99038 466.824,9.91386 470.404,9.83865 473.984,9.76472 477.564,9.69203 481.144,9.62054 484.724,9.55024 
 488.304,9.48109 491.885,9.41307 495.465,9.34614 499.045,9.28028 502.625,9.21547 506.205,9.15168 509.785,9.08888 513.365,9.02706 516.945,8.96618 520.525,8.90624 
 524.105,8.8472 527.685,8.78905 531.265,8.73177 534.845,8.67534 538.425,8.61973 542.005,8.56494 545.585,8.51094 549.165,8.45772 552.745,8.40526 556.325,8.35354 
 559.906,8.30255 563.486,8.25227 567.066,8.20269 570.646,8.1538 574.226,8.10557 577.806,8.058 581.386,8.01107 584.966,7.96477 588.546,7.91909 592.126,7.87402 
 
 "/>
 
 
 
 
 BE Sum

In [28]:
# meV / K / unit cell --> J / K / mol
# OK; so factor of /1000 for meV -> eV
# factor of q for eV -> J
# factor of Na + 12(atoms) for moles?
Na=6.022E23

#1.1*q/1000*Na*12

SHCsSI=SHCs.*q/1000*Na*1

151-element Array{Float64,1}:
 NaN     
  16.7813
  40.1088
  56.9009
  68.7047
  77.2185
  83.5732
  88.4694
  92.3454
  95.4842
  98.0751
 100.248 
 102.097 
   ⋮     
 122.594 
 122.609 
 122.624 
 122.638 
 122.653 
 122.667 
 122.681 
 122.695 
 122.708 
 122.722 
 122.735 
 122.748 

In [29]:
using Plots
plot(Ts,SHCsSI,lab="BE Sum")
plot!(ylim=[0,Inf])
plot!(ylab="Specific Heat Capacity (J/K/mol (unitcell))")
plot!(xlab="Temperature (K)")

#savefig("SHC_vs_T.png")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 3000 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 Temperature (K) 
 
 
 Specific Heat Capacity (J/K/mol (unitcell)) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 58.6982,322.6 62.2782,253.316 65.8583,203.443 69.4383,168.385 73.0184,143.099 76.5984,124.225 80.1785,109.683 83.7585,98.1714 87.3386,88.8488 90.9186,81.1539 
 94.4987,74.6992 98.0787,69.2101 101.659,64.4865 105.239,60.3797 108.819,56.7771 112.399,53.5916 115.979,50.755 119.559,48.2132 123.139,45.9226 126.719,43.8479 
 130.299,41.96 133.879,40.2349 137.459,38.6523 141.039,37.1954 144.619,35.8498 148.199,34.6033 151.78,33.4452 155.36,32.3665 158.94,31.3594 162.52,30.4169 
 166.1,29.5331 169.68,28.7026 173.26,27.9207 176.84,27.1833 180.42,26.4867 184,25.8277 187.58,25.2032 191.16,24.6106 194.74,24.0476 198.32,23.5119 
 201.9,23.0017 205.48,22.5151 209.06,22.0506 212.64,21.6067 216.22,21.1821 219.801,20.7754 223.381,20.3857 226.961,20.0119 230.541,19.6529 234.121,19.308 
 237.701,18.9764 241.281,18.6572 244.861,18.3499 248.441,18.0537 252.021,17.768 255.601,17.4924 259.181,17.2262 262.761,16.9691 266.341,16.7205 269.921,16.4801 
 273.501,16.2474 277.081,16.022 280.661,15.8037 284.241,15.5921 287.822,15.387 291.402,15.1879 294.982,14.9947 298.562,14.807 302.142,14.6248 305.722,14.4476 
 309.302,14.2754 312.882,14.1079 316.462,13.9449 320.042,13.7862 323.622,13.6317 327.202,13.4813 330.782,13.3346 334.362,13.1917 337.942,13.0524 341.522,12.9165 
 345.102,12.7839 348.682,12.6545 352.262,12.5281 355.843,12.4048 359.423,12.2843 363.003,12.1666 366.583,12.0515 370.163,11.9391 373.743,11.8291 377.323,11.7215 
 380.903,11.6163 384.483,11.5134 388.063,11.4126 391.643,11.314 395.223,11.2174 398.803,11.1228 402.383,11.0301 405.963,10.9393 409.543,10.8503 413.123,10.7631 
 416.703,10.6775 420.283,10.5937 423.864,10.5114 427.444,10.4307 431.024,10.3516 434.604,10.2739 438.184,10.1977 441.764,10.1228 445.344,10.0493 448.924,9.97715 
 452.504,9.90628 456.084,9.83666 459.664,9.76825 463.244,9.70104 466.824,9.63499 470.404,9.57006 473.984,9.50623 477.564,9.44348 481.144,9.38177 484.724,9.32108 
 488.304,9.26139 491.885,9.20266 495.465,9.14488 499.045,9.08803 502.625,9.03207 506.205,8.977 509.785,8.92279 513.365,8.86942 516.945,8.81687 520.525,8.76512 
 524.105,8.71415 527.685,8.66395 531.265,8.6145 534.845,8.56579 538.425,8.51778 542.005,8.47048 545.585,8.42387 549.165,8.37792 552.745,8.33263 556.325,8.28798 
 559.906,8.24396 563.486,8.20056 567.066,8.15776 570.646,8.11555 574.226,8.07391 577.806,8.03285 581.386,7.99233 584.966,7.95237 588.546,7.91293 592.126,7.87402 
 
 "/>
 
 
 
 
 BE Sum

In [30]:
#Characteristic energy
f=2.25E12 # 2.25 THz - see Polaron paper, http://arxiv.org/abs/1704.05404 and references therein
ω=2*π*f
E=ħ*ω
println("Characteristic phonon energy: $f Hz = ",E/q," eV")

Characteristic phonon energy: 2.25e12 Hz = 0.009305251500657504 eV


In [31]:
# optical phonon scattering rate
τ=0.12 # in ps
println("Initial energy loss to scattering into LO modes, ",(E/q)/τ," eV/ps") # in eV/ps

Initial energy loss to scattering into LO modes, 0.0775437625054792 eV/ps


In [32]:
# For paper --- calculation of polaron temperature, assuming modal occupation as above
cells=360 # from volume Polaron, r=27.4 Angstrom
SHC=1.25E-3 # meV/K, from rough of above table
E=1.75 # maximum of energy in eV

T=E/(SHC*cells)

3.888888888888889

In [33]:
# Should really use unitful here!

ω=2.25E12
meff=0.12

shm=sqrt((ħ*ω)^2/(meff*MassElectron))
#sqrt(ω^2*(meff*me))

7.176679038355128e-7

In [34]:
# Overtones in MAPI Polarons
# These are the Franck-Condon vibration states

# Schultz: 
#phononfreq= 17.1009 #(int units) = 3.8477e+13 [SI, Hz]
#electronfreq= 10.3716 #(int units) = 2.33361e+13 [SI, Hz]
#combinedfreq= 8.86807 #(int units) = 1.99532e+13 [SI, Hz]
#println("Energy: (v=$v, f=$f Hz) = ",1000*v*E/q," meV")

#Characteristic energy
v=20
f=2.25E12 # 2.25 THz

function FC_overtone(;v=20,f=2.25E12)
    ω=2*π*f
    E=ħ*ω
    println("Franck condon overtone energy: (v=$v, f=$f Hz) = ",1000*v*E/q," meV")
    return(E)
end

println("From finite Temperature T=300 variational solution for MAPI (Polaron paper)")
FC_overtone(v=20,f=2.25E12)
FC_overtone(v=20,f=1.0E12)

println("From Feynman small-α approximation, athermal variational solution, for α=$(α). ")
w=3 ; v=w+(2/9)*α
FC_overtone(v=v,f=2.25E12)
FC_overtone(v=v,f=1.0E12)

From finite Temperature T=300 variational solution for MAPI (Polaron paper)
Franck condon overtone energy: (v=20, f=2.25e12 Hz) = 186.10503001315007 meV
Franck condon overtone energy: (v=20, f=1.0e12 Hz) = 82.71334667251114 meV
From Feynman small-α approximation, athermal variational solution, for α=2.6765070822960193. 
Franck condon overtone energy: (v=3.5947793516213373, f=2.25e12 Hz) = 33.45032595620706 meV
Franck condon overtone energy: (v=3.5947793516213373, f=1.0e12 Hz) = 14.866811536092026 meV


6.626068959988852e-22

In [35]:
α=feynmanalpha(ε_Inf, ε_S ,  f,    meff)
println("\nMAPI(meff=$meff): α=$α")

scale=sqrt(2*meff*melectron*ω )
w=3 ; v=w+(2/9)*α
println("v=$v w=$w by the Feynman1955 small-α expansion")


#v=20.0; w=1.15

mu=μ=((v^2-w^2)/v^2)

# (2.4)
rf=sqrt(3/(2*mu*v))
# (2.4) SI scaling inferred from units in (2.5a) and Table II
rfsi=rf*sqrt(2*me*ω )
@printf("\n Schultz1959(2.4): rf= %g (int units) = %g m [SI]",rf,rfsi )



MAPI(meff=0.12): α=2.3939407113094093
v=3.5319868247354242 w=3 by the Feynman1955 small-α expansion

 Schultz1959(2.4): rf= 1.23476 (int units) = 8.66011e-10 m [SI]

In [36]:
# Let's cross check this by visualising!

# Schultz1959 - above (2.4) - SHM wave function
ψ(r)=(μ*v/π)^(3/4) * exp(-μ*v*r^2/2)

rs=-rf*3:rf/100:rf*3
y=[ψ(r) for r in rs]

using Plots
gr()

plot(rs,y,label="Polaron Wavefunction") # fmt=:png)
#plot!(x,y.^2,label="Polaron Wavefunction ^2")
plot!([0-rf,0+rf],[ψ(0),ψ(0)],label="Polaron Radius",fill=(0,:red),fillalpha=0.2)
plot!(xlab="Radius (internal units)")
plot!(ylab="Wavefunction")



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 Radius (internal units) 
 
 
 Wavefunction 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.1181,372.014 56.0131,371.994 56.9081,371.974 57.8031,371.953 58.6982,371.931 59.5932,371.907 60.4882,371.883 61.3832,371.858 62.2782,371.832 63.1732,371.805 
 64.0682,371.776 64.9633,371.747 65.8583,371.716 66.7533,371.684 67.6483,371.651 68.5433,371.617 69.4383,371.581 70.3333,371.543 71.2283,371.504 72.1234,371.464 
 73.0184,371.422 73.9134,371.378 74.8084,371.333 75.7034,371.286 76.5984,371.237 77.4934,371.186 78.3885,371.134 79.2835,371.079 80.1785,371.022 81.0735,370.963 
 81.9685,370.902 82.8635,370.838 83.7585,370.772 84.6535,370.704 85.5486,370.633 86.4436,370.56 87.3386,370.484 88.2336,370.405 89.1286,370.323 90.0236,370.238 
 90.9186,370.151 91.8136,370.06 92.7087,369.966 93.6037,369.868 94.4987,369.767 95.3937,369.663 96.2887,369.555 97.1837,369.443 98.0787,369.327 98.9738,369.207 
 99.8688,369.083 100.764,368.955 101.659,368.823 102.554,368.686 103.449,368.545 104.344,368.399 105.239,368.248 106.134,368.092 107.029,367.93 107.924,367.764 
 108.819,367.592 109.714,367.415 110.609,367.232 111.504,367.043 112.399,366.848 113.294,366.647 114.189,366.439 115.084,366.226 115.979,366.005 116.874,365.778 
 117.769,365.543 118.664,365.301 119.559,365.053 120.454,364.796 121.349,364.532 122.244,364.26 123.139,363.98 124.034,363.691 124.929,363.394 125.824,363.089 
 126.719,362.774 127.614,362.45 128.509,362.118 129.404,361.775 130.299,361.423 131.194,361.061 132.089,360.689 132.984,360.307 133.879,359.914 134.774,359.51 
 135.669,359.095 136.564,358.669 137.459,358.231 138.354,357.782 139.249,357.321 140.144,356.848 141.039,356.362 141.934,355.864 142.829,355.352 143.724,354.828 
 144.619,354.29 145.514,353.739 146.409,353.174 147.304,352.594 148.199,352.001 149.094,351.392 149.99,350.769 150.885,350.131 151.78,349.477 152.675,348.808 
 153.57,348.123 154.465,347.422 155.36,346.704 156.255,345.97 157.15,345.219 158.045,344.451 158.94,343.666 159.835,342.863 160.73,342.042 161.625,341.203 
 162.52,340.346 163.415,339.47 164.31,338.575 165.205,337.661 166.1,336.728 166.995,335.775 167.89,334.803 168.785,333.811 169.68,332.798 170.575,331.765 
 171.47,330.711 172.365,329.637 173.26,328.541 174.155,327.424 175.05,326.286 175.945,325.126 176.84,323.944 177.735,322.739 178.63,321.513 179.525,320.264 
 180.42,318.993 181.315,317.699 182.21,316.382 183.105,315.041 184,313.678 184.895,312.291 185.79,310.881 186.685,309.447 187.58,307.99 188.475,306.508 
 189.37,305.003 190.265,303.473 191.16,301.92 192.055,300.342 192.95,298.74 193.845,297.114 194.74,295.464 195.635,293.789 196.53,292.089 197.425,290.366 
 198.32,288.618 199.215,286.845 200.11,285.048 201.005,283.227 201.9,281.382 202.795,279.512 203.69,277.618 204.585,275.7 205.48,273.758 206.375,271.792 
 207.27,269.803 208.165,267.79 209.06,265.753 209.955,263.693 210.85,261.61 211.745,259.503 212.64,257.374 213.535,255.223 214.43,253.049 215.325,250.853 
 216.22,248.636 217.115,246.396 218.01,244.136 218.906,241.855 219.801,239.552 220.696,237.23 221.591,234.888 222.486,232.526 223.381,230.145 224.276,227.745 
 225.171,225.327 226.066,222.89 226.961,220.436 227.856,217.965 228.751,215.478 229.646,212.974 230.541,210.455 231.436,207.92 232.331,205.371 233.226,202.808 
 234.121,200.232 235.016,197.642 235.911,195.04 236.806,192.427 237.701,189.802 238.596,187.167 239.491,184.522 240.386,181.868 241.281,179.205 242.176,176.535 
 243.071,173.857 243.966,171.173 244.861,168.484 245.756,165.789 246.651,163.09 247.546,160.388 248.441,157.683 249.336,154.977 250.231,152.269 251.126,149.561 
 252.021,146.853 252.916,144.147 253.811,141.443 254.706,138.742 255.601,136.045 256.496,133.352 257.391,130.665 258.286,127.985 259.181,125.312 260.076,12

In [37]:
savefig("polaron-radius.png")

# needs to be in a cell by itself as otherwise it breaks the inline output above

In [38]:
plot(rs,y.^2,label="Polaron Wavefn.^2")
plot!([0-rf,0+rf],[ψ(0).^2,ψ(0).^2],label="Polaron Radius",fill=(0,:red),fillalpha=0.2)



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,376.377 40.2913,376.377 41.2126,376.377 42.1339,376.377 43.0551,376.377 43.9764,376.377 44.8976,376.377 45.8189,376.377 46.7402,376.377 47.6614,376.377 
 48.5827,376.377 49.5039,376.377 50.4252,376.376 51.3465,376.376 52.2677,376.376 53.189,376.376 54.1102,376.376 55.0315,376.376 55.9528,376.376 56.874,376.375 
 57.7953,376.375 58.7165,376.375 59.6378,376.375 60.5591,376.374 61.4803,376.374 62.4016,376.374 63.3228,376.373 64.2441,376.373 65.1654,376.372 66.0866,376.372 
 67.0079,376.371 67.9291,376.371 68.8504,376.37 69.7717,376.37 70.6929,376.369 71.6142,376.368 72.5354,376.367 73.4567,376.366 74.378,376.366 75.2992,376.365 
 76.2205,376.363 77.1417,376.362 78.063,376.361 78.9843,376.36 79.9055,376.358 80.8268,376.357 81.748,376.355 82.6693,376.353 83.5906,376.351 84.5118,376.349 
 85.4331,376.347 86.3543,376.344 87.2756,376.342 88.1969,376.339 89.1181,376.336 90.0394,376.333 90.9606,376.329 91.8819,376.326 92.8031,376.322 93.7244,376.317 
 94.6457,376.313 95.5669,376.308 96.4882,376.303 97.4094,376.297 98.3307,376.291 99.252,376.285 100.173,376.278 101.094,376.271 102.016,376.263 102.937,376.255 
 103.858,376.246 104.78,376.237 105.701,376.227 106.622,376.216 107.543,376.205 108.465,376.192 109.386,376.179 110.307,376.166 111.228,376.151 112.15,376.135 
 113.071,376.119 113.992,376.101 114.913,376.082 115.835,376.063 116.756,376.041 117.677,376.019 118.598,375.995 119.52,375.97 120.441,375.943 121.362,375.914 
 122.283,375.884 123.205,375.852 124.126,375.818 125.047,375.782 125.969,375.744 126.89,375.704 127.811,375.661 128.732,375.616 129.654,375.568 130.575,375.518 
 131.496,375.465 132.417,375.408 133.339,375.349 134.26,375.286 135.181,375.22 136.102,375.15 137.024,375.076 137.945,374.998 138.866,374.916 139.787,374.829 
 140.709,374.738 141.63,374.642 142.551,374.541 143.472,374.435 144.394,374.323 145.315,374.206 146.236,374.082 147.157,373.952 148.079,373.816 149,373.672 
 149.921,373.522 150.843,373.364 151.764,373.198 152.685,373.024 153.606,372.842 154.528,372.651 155.449,372.45 156.37,372.24 157.291,372.021 158.213,371.791 
 159.134,371.55 160.055,371.298 160.976,371.035 161.898,370.759 162.819,370.471 163.74,370.171 164.661,369.857 165.583,369.529 166.504,369.187 167.425,368.83 
 168.346,368.457 169.268,368.069 170.189,367.665 171.11,367.243 172.031,366.804 172.953,366.347 173.874,365.871 174.795,365.376 175.717,364.861 176.638,364.325 
 177.559,363.768 178.48,363.19 179.402,362.589 180.323,361.965 181.244,361.318 182.165,360.646 183.087,359.949 184.008,359.226 184.929,358.477 185.85,357.701 
 186.772,356.897 187.693,356.064 188.614,355.202 189.535,354.311 190.457,353.388 191.378,352.434 192.299,351.448 193.22,350.43 194.142,349.378 195.063,348.291 
 195.984,347.17 196.906,346.013 197.827,344.819 198.748,343.589 199.669,342.32 200.591,341.014 201.512,339.668 202.433,338.282 203.354,336.856 204.276,335.389 
 205.197,333.88 206.118,332.329 207.039,330.735 207.961,329.097 208.882,327.416 209.803,325.689 210.724,323.918 211.646,322.101 212.567,320.238 213.488,318.328 
 214.409,316.372 215.331,314.368 216.252,312.316 217.173,310.216 218.094,308.068 219.016,305.872 219.937,303.626 220.858,301.332 221.78,298.988 222.701,296.596 
 223.622,294.154 224.543,291.662 225.465,289.122 226.386,286.532 227.307,283.892 228.228,281.204 229.15,278.468 230.071,275.682 230.992,272.849 231.913,269.968 
 232.835,267.039 233.756,264.064 234.677,261.042 235.598,257.974 236.52,254.861 237.441,251.704 238.362,248.503 239.283,245.259 240.205,241.974 241.126,238.647 
 242.047,235.28 242.969,231.875 243.89,228.431 244.811,224.951 245.732,221.436 246.654,217.886 247.575,214.304 248.496,210.691 249.417,207.048 250.339,203.376 
 251.26,199.678 252.181,195.

## A note on Schultz

In Schultz1959 (https://doi.org/10.1103%2Fphysrev.116.526), a Feynman polaron radius is defined from the Gaussian form of the Simple Harmonic Oscillator specified by the Feynman solution to the polaron problem.

The reduced mass is defined in terms of the internal parameters $v$ and $w$,

$\mu=m(v^2-w^2)/v^2$.

The (Gaussian) wavefunction is specified (plotted above),

$\psi(r)=(\mu v/\pi)^{\frac{3}{4}} exp(-\frac{\mu v r^2}{2})$.

And from considering the standard deviation of this wavefunction, a radius is defined: 

$r_f \equiv (<p^2>)^{\frac{1}{2}} = (3/2\mu v)^{\frac{1}{2}}$ (2.4, Schultz)

Here $p$ is the momentum of the system, $\mu$ is the reduced (effective) mass of the electron and interacting phonon-cloud, while $v$ and $w$ are internal polaron parameters characterising the harmonic motion of the polaron.
The units of $v$ and $w$ are $\hbar\omega$.

"Using the weak... coupling expansions given by Feynman for $w$ and $v$, we find"

$r_f (\alpha\rightarrow 0) \approx  (3/0.44\alpha)^{\frac{1}{2}} (2m\omega)^{-\frac{1}{2}}$

To understand where this magic number (0.44) comes from, you must return to Feynman1955 and repeat the derivation used to get the small-$\alpha$ expansion for energy. This is the part around Eqn 35 to 36 in Feynman 1955. 

Finding the term $2\mu v$ is our aim.

Following Feynman, assuming $\alpha$ is small, $v=(1+\epsilon)w$. 
You can then go back to the energy integral Feynman(31), substituting your $v$ expansion in. 
You can expand the square-root on on the denominator, by pulling out terms of $w$ to simplify, before taking a term of $\tau^{\frac{1}{2}}$ out to place the root in the form of (1+x)^n. You can then use a binomial expansion for this quare root, and keep only zeroth and first order in $\epsilon$ terms. The resulting integral in the total energy has two parts, the zeroth-order contribution integrating to 1, and the linear in $\epsilon$ term (P) apparently analytic and provided by Feynman (35),

$2w^{-1} [ (1+w)^{\frac{1}{2}} - 1] = P$.

This makes the total energy,  

$E=\frac{3}{4v}(v-w)^2-A $, (Feynman33) 

which with our linear-in-$\epsilon$ form of the integral $A$,

$E = \frac{3}{4v}(v-w)^2 - \alpha \frac{v}{w} [1-P] $

Again, substituting $v=(1+\epsilon)w$, 

$E=\frac{3}{4}\frac{((1+\epsilon)w - w)^2}{(1+\epsilon)w} - \alpha\frac{(1+\epsilon)w}{w} [1-P]$,

and discarding high order (in $\epsilon$) terms,

$E = \frac{3}{4} w\epsilon^2 -\alpha -\alpha\epsilon[1-P]$, 

as given by Feynman. 

If we reorder terms,

$E = \frac{3}{4} w\epsilon^2 -\alpha\epsilon[1-P]-(\alpha)$, 

it becomes clear the total energy has a quadratic ($E =a\epsilon^2 + b\epsilon +c $) form in $\epsilon$. 

We will want a variational (minimum energy) solution. This found at the minimum, $x=\frac{-b}{2a}$ in the standard quadratic formula, and so, 

$\epsilon = \frac{2}{3} \alpha \frac{(1-P)}{w}$. 

This is as given by Feynman, valid for small-$\alpha$ only.

We can now return to the key $2\mu v$ term.

Expanding this identity with our definition of $\mu$,

$2\mu v = 2m_e \frac{v^2-w^2}{v}$.

Substituting in $v=(1+\epsilon)w$, following through the algebra, and as epsilon is small approximating $\frac{1}{1+\epsilon}=\frac{1}{1}$, we eventually have

$2\mu v = 2m_e[2w\epsilon]$.

Substituting in our form for $\epsilon$, by simple algebraic rearrangement we get

$v=(1+\frac{2\alpha(1-P)}{3w})w = w + \frac{2}{3} \alpha (1-P)$

Feynman states that 'the variational solution is least for $w=3$', and goes on to state that the energy correction is anyway insensitive to the choice of $w$.

If $w=3$, $P=\frac{2}{3}$, and we have that,

$v=3+\frac{2}{9}\alpha$.

Thus, $2\mu v = 2m_e \frac{4}{9}\alpha = 2m_e 0.\overline{4} \alpha$.

Schultz1959 Eqn. 2.5a approximates this as $0.44$. It is not obvious whether this arrises as a printing error, or was an intentional (1% error) approximation. 

It is also not clear in Schultz where the table given (reproduced above) comes from. 
Neither of the provided identities gives it. 
Possibly these data come from a direct numerical integration of $A$ and a variational solution for the parameter $v$ and $w$, using Feynman(33) as the variational energy. 
Certainly it would be worth adding this athermal energy to the (more complex) machinery at https://github.com/jarvist/PolaronMobility.jl . 